# 📊 Analyse Exploratoire des Pals (Palworld Dataset)Ce notebook répond aux questions a → x de l'analyse exploratoire à partir des fichiers nettoyés.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 📁 Chargement des fichiers nettoyés
combat_df = pd.read_csv('../2_cleaned/combat_attribute_clean.csv')
job_df = pd.read_csv('../2_cleaned/job_skill_clean.csv')

# 🔤 Normalisation des noms de colonnes
combat_df.columns = combat_df.columns.str.strip().str.lower().str.replace(' ', '_')
job_df.columns = job_df.columns.str.strip().str.lower().str.replace(' ', '_')

# 🔗 Fusion des datasets sur le nom
combat_df['name'] = combat_df['name'].str.strip().str.lower()
job_df['english_name'] = job_df['english_name'].str.strip().str.lower()
merged_df = pd.merge(combat_df, job_df, left_on='name', right_on='english_name', how='inner')

## a. Distribution de la taille des Pals (`volume_size`)

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=merged_df, x='volume_size_x', order=merged_df['volume_size_x'].value_counts().index)
plt.title('a. Distribution de la taille des Pals')
plt.xlabel('Taille (volume_size)')
plt.ylabel('Nombre de Pals')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## b. Distribution des catégories (`genuscategory`)

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=merged_df, x='genuscategory', order=merged_df['genuscategory'].value_counts().index)
plt.title('b. Distribution des catégories des Pals (genuscategory)')
plt.xlabel('Catégorie biologique')
plt.ylabel('Nombre de Pals')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## c. Distribution des points de vie (HP)

In [ ]:
plt.figure(figsize=(10, 5))sns.histplot(data=merged_df, x='hp', bins=20, kde=True)plt.title('c. Distribution des points de vie (HP)')plt.xlabel('Points de vie (HP)')plt.ylabel('Nombre de Pals')plt.tight_layout()plt.show()

## d. Distribution de la rareté (rarity)

In [ ]:
plt.figure(figsize=(10, 5))sns.countplot(data=merged_df, x='rarity', order=sorted(merged_df['rarity'].dropna().unique()))plt.title("d. Distribution de la rareté des Pals")plt.xlabel("Rareté")plt.ylabel("Nombre de Pals")plt.tight_layout()plt.show()

## e. Distribution de la consommation alimentaire (food_intake)

In [ ]:
plt.figure(figsize=(10, 5))sns.countplot(data=merged_df, x='food_intake', order=sorted(merged_df['food_intake'].dropna().unique()))plt.title("e. Distribution de la consommation alimentaire des Pals")plt.xlabel("Food Intake")plt.ylabel("Nombre de Pals")plt.tight_layout()plt.show()

## f. Pals utiles à la production (ranch : laine, œuf, lait…)

In [ ]:
ranch_items = merged_df[merged_df['ranch_items'].notna()][['name', 'ranch_items']]print("Pals utiles à la production (ranch items):")print(ranch_items.sort_values('ranch_items'))

## g. Top 10 des Pals les plus puissants (par attaque et défense combinées)

In [ ]:
merged_df['total_power'] = merged_df[['melee_attack', 'remote_attack', 'defense']].sum(axis=1, numeric_only=True)top_powerful = merged_df[['name', 'total_power']].sort_values(by='total_power', ascending=False).head(10)print("Top 10 des Pals les plus puissants :")print(top_powerful)

## h. Corrélations entre les attributs de combat

In [ ]:
combat_stats = merged_df[['hp', 'melee_attack', 'remote_attack', 'defense', 'support', 'speed_of_work']].dropna()plt.figure(figsize=(10, 6))sns.heatmap(combat_stats.corr(), annot=True, cmap="coolwarm")plt.title("h. Corrélation entre les attributs de combat")plt.tight_layout()plt.show()

## i. Impact de la rareté sur les attributs de base (HP, attaque, défense)

In [ ]:
plt.figure(figsize=(10, 6))sns.boxplot(data=merged_df, x='rarity', y='hp')plt.title("i. HP en fonction de la rareté")plt.tight_layout()plt.show()plt.figure(figsize=(10, 6))sns.boxplot(data=merged_df, x='rarity', y='melee_attack')plt.title("i. Melee Attack en fonction de la rareté")plt.tight_layout()plt.show()

## j. Rareté moyenne des Pals à forte attaque

In [ ]:
strong_pals = merged_df[merged_df['melee_attack'] >= merged_df['melee_attack'].quantile(0.9)]moyenne_rareté = strong_pals['rarity'].mean()print(f"Rareté moyenne des Pals avec attaque élevée (top 10%) : {moyenne_rareté:.2f}")

## k. Influence de la taille sur la performance au combat

In [ ]:
plt.figure(figsize=(10, 6))sns.boxplot(data=merged_df, x='volume_size_x', y='total_power')plt.title("k. Influence de la taille sur la puissance totale")plt.xlabel("Taille")plt.ylabel("Puissance totale (attaque + défense)")plt.tight_layout()plt.show()

## l. Les Pals rapides sont-ils plus efficaces ?

In [ ]:
plt.figure(figsize=(10, 6))sns.scatterplot(data=merged_df, x='running_speed', y='total_power')plt.title("l. Vitesse vs. Puissance totale")plt.xlabel("Vitesse de course")plt.ylabel("Puissance totale")plt.tight_layout()plt.show()

## m. Proposer une équipe équilibrée (5 Pals aux attributs complémentaires)

In [ ]:
balanced_team = merged_df.sort_values(by=['total_power', 'speed_of_work'], ascending=False).drop_duplicates(subset=['genuscategory']).head(5)print("m. Équipe équilibrée de 5 Pals :")print(balanced_team[['name', 'total_power', 'hp', 'melee_attack', 'defense', 'speed_of_work', 'genuscategory']])

## n. Compétences de travail les plus répandues

In [ ]:
work_skills = ['make_a_fire', 'watering', 'planting', 'generate_electricity', 'manual', 'collection', 'logging', 'mining', 'pharmaceutical', 'cool_down', 'pasture', 'carry']skill_counts = job_df[work_skills].sum().sort_values(ascending=False)print("n. Compétences de travail les plus répandues :")print(skill_counts)

## o. Compétences de travail les moins répandues

In [ ]:
print("o. Compétences de travail les moins répandues :")print(skill_counts[::-1])

## p. Nombre de Pals adaptés au travail de nuit

In [ ]:
night_workers = merged_df[merged_df['night_shift'].notna()]print(f"p. Nombre de Pals adaptés au travail de nuit : {len(night_workers)}")

## q. Traits communs des Pals de travail de nuit

In [ ]:
print("q. Extraits des traits des Pals de nuit :")print(night_workers[['name', 'rarity', 'food_intake', 'total_skills', 'handling_speed']].head())

## r. Rareté moyenne des Pals les plus polyvalents (plus de compétences)

In [ ]:
versatile_pals = merged_df.sort_values(by='total_skills', ascending=False).head(10)mean_rarity = versatile_pals['rarity'].mean()print(f"r. Rareté moyenne des Pals les plus polyvalents : {mean_rarity:.2f}")

## s. Pals avec la vitesse de travail la plus élevée

In [ ]:
fast_workers = merged_df[['name', 'handling_speed']].sort_values(by='handling_speed', ascending=False).dropna().head(10)print("s. Pals avec la vitesse de travail la plus élevée :")print(fast_workers)

## t. Pals les plus faciles à capturer + stratégie

In [ ]:
easy_catch = merged_df[['name', 'catch_rate']].sort_values(by='catch_rate', ascending=False).dropna().head(10)print("t. Pals les plus faciles à capturer :")print(easy_catch)print("\n💡 Stratégie : commencer par ceux à fort taux de capture pour constituer une base efficace rapidement.")

## u. Boss avec attributs de combat les plus forts (total des stats)

In [ ]:
boss_df = pd.read_csv('../2_cleaned/ordinary_boss_clean.csv')boss_df.columns = boss_df.columns.str.strip().str.lower().str.replace(" ", "_")boss_df['combat_score'] = boss_df[['hp', 'remote_attack']].sum(axis=1, numeric_only=True)strongest_boss = boss_df.sort_values(by='combat_score', ascending=False).head(1)print("u. Boss avec le score de combat le plus élevé :")print(strongest_boss)

## v. Répartition des niveaux d’apparition

In [ ]:
refresh_df = pd.read_csv('../2_cleaned/refresh_level_clean.csv')refresh_df.columns = refresh_df.columns.str.strip().str.lower().str.replace(" ", "_")plt.figure(figsize=(10, 5))sns.histplot(data=refresh_df, x='minimum_level', bins=15, kde=True)plt.title("v. Répartition des niveaux d'apparition (minimum_level)")plt.xlabel("Niveau minimum")plt.ylabel("Nombre de Pals")plt.tight_layout()plt.show()

## w. Répartition des zones d’apparition

In [ ]:
plt.figure(figsize=(12, 6))sns.countplot(data=refresh_df, x='refresh_area', order=refresh_df['refresh_area'].value_counts().index)plt.title("w. Répartition des zones d’apparition")plt.xlabel("Zone")plt.ylabel("Nombre de Pals")plt.xticks(rotation=45)plt.tight_layout()plt.show()

## x. Autres observations possibles

In [ ]:
# Exemple : relation entre prix et raretéplt.figure(figsize=(10, 5))sns.boxplot(data=merged_df, x='rarity', y='price')plt.title("x. Prix des Pals en fonction de leur rareté")plt.xlabel("Rareté")plt.ylabel("Prix")plt.tight_layout()plt.show()